In [ ]:
from pynq import Overlay, allocate, PL

PL.reset()
overlay = Overlay('wahwah.bit')

In [ ]:
print('IP blocks :', list(overlay.ip_dict.keys()))

In [ ]:
wahwah_filter     = overlay.wah_wah_filter_axi_0
dma_0 = overlay.axi_dma_0
dma_0_send = overlay.axi_dma_0.sendchannel
dma_0_recv = overlay.axi_dma_0.recvchannel

CONTROL_REGISTER = 0x0

def get_register_offset(overlay, ip, parameter):
    return overlay.ip_dict[ip]['registers'][parameter]['address_offset']
        
SAMPLE_RATE_REGISTER = get_register_offset(overlay, 'wah_wah_filter_axi_0', 'sample_rate')
MIN_FREQ_REGISTER = get_register_offset(overlay, 'wah_wah_filter_axi_0', 'min_freq')
MAX_FREQ_REGISTER = get_register_offset(overlay, 'wah_wah_filter_axi_0', 'max_freq')
LFO_FREQ_REGISTER = get_register_offset(overlay, 'wah_wah_filter_axi_0', 'lfo_freq')

wahwah_filter.write(SAMPLE_RATE_REGISTER, 48000)
wahwah_filter.write(MIN_FREQ_REGISTER, 500.0)
wahwah_filter.write(MAX_FREQ_REGISTER, 3000.0)
wahwah_filter.write(LFO_FREQ_REGISTER, 0.2)

In [ ]:
def process_buffers_fpga(in_buffer, out_buffer):

    wahwah_filter.write(CONTROL_REGISTER, 0x01)
    
    dma_0_send.transfer(in_buffer)
    dma_0_recv.transfer(out_buffer)
    dma_0_send.wait()
    dma_0_recv.wait()

def process_fpga(raw_data):
    
    in_buffer = allocate(shape=(len(raw_data),), dtype=raw_data.dtype)
    out_buffer = allocate(shape=(len(raw_data),), dtype=raw_data.dtype)
    
    process_buffers_fpga(in_buffer, out_buffer)
    
    return out_buffer

In [ ]:
from IPython.display import Audio as IPAudio
IPAudio("recording_0.wav")

## Plotting PCM data

Users can display the audio data in notebook:

1. Plot the audio signal's amplitude over time.
2. Plot the spectrogram of the audio signal.

The next cell reads the saved audio file and processes it into a `numpy` array.
Note that if the audio sample width is not standard, additional processing
is required. In the following example, the `sample_width` is read from the
wave file itself (24-bit dual-channel PCM audio, where `sample_width` is 3 bytes).

In [ ]:
%matplotlib inline
import wave
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft

wav_path = "recording_0.wav"
with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()

    #print("in data type: ", raw_frames.dtype, " len: ", len(raw_frames))
    print("num_frames: ", num_frames)
    print("num_channels: ", num_channels)
    print("sample_rate: ", sample_rate)
    print("sample_width: ", sample_width)
        
temp_buffer = np.empty((num_frames, num_channels, 4), dtype=np.uint8)
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, 
                                                    sample_width)
temp_buffer[:, :, sample_width:] = \
    (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 255
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])

### 1. Amplitude over time

In [ ]:
for channel_index in range(num_channels):
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Audio in Time Domain (Channel {})'.format(channel_index))
    plt.xlabel('Time in s')
    plt.ylabel('Amplitude')
    time_axis = np.arange(0, num_frames/sample_rate, 1/sample_rate)
    plt.plot(time_axis, frames[:, channel_index])
    plt.show()

In [ ]:
filtered_data = []
for i in range(num_channels):
    filtered = process_fpga(frames[:, 0])
    filtered_data.append(filtered)

In [ ]:
for channel_index in range(num_channels):
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Audio in Time Domain (Channel {})'.format(channel_index))
    plt.xlabel('Time in s')
    plt.ylabel('Amplitude')
    time_axis = np.arange(0, num_frames/sample_rate, 1/sample_rate)
    plt.plot(time_axis, filtered_data[channel_index])
    plt.show()